In [1]:
import sys
import geopandas as gpd
import fiona
from shapely.geometry import Polygon
from shapely.ops import transform
import math
import pyproj
sys.path.append('..')
from h3_helper import *
from s2_helper import *
from rhealpix_helper import *
from eaggr_helper import *
import pywraps2 as s2
#from dggrid4py import DGGRIDv7, Dggs, dgselect, dggs_types

In [11]:
# Area from individual lambert azimutal projection
def get_lambert_area(geom):
    '''Area from cell's lambert azimutal projection'''
    if (-180 <= geom.centroid.x <= 180) and (-90 <= geom.centroid.y <= 90):
        proj_str = f"+proj=laea +lat_0={geom.centroid.y} +lon_0={geom.centroid.x}"
        project = pyproj.Transformer.from_crs('EPSG:4236', proj_str, always_xy=True).transform
        area = transform(project, geom).area
        #perimeter = transform(project, geom).length
    else:
        print(f'invalid centroid {geom.centroid}')
        area = None
        #perimeter = None
    return area

In [15]:
def get_perimeter_from_lambert(geom):
    '''Area from cell's lambert azimutal projection'''
    if (-180 <= geom.centroid.x <= 180) and (-90 <= geom.centroid.y <= 90):
        proj_str = f"+proj=laea +lat_0={geom.centroid.y} +lon_0={geom.centroid.x}"
        project = pyproj.Transformer.from_crs('EPSG:4236', proj_str, always_xy=True).transform
        perimeter = transform(project, geom).length
    else:
        print(f'invalid centroid {geom.centroid}')
        perimeter = None
    return perimeter
    

In [6]:
# Area from UTM zone
def convert_wgs_to_utm(lon, lat):
    utm_band = str((math.floor((lon + 180) / 6 ) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0'+utm_band
    if lat >= 0:
        epsg_code = '326' + utm_band
    else:
        epsg_code = '327' + utm_band
    return epsg_code

def get_utm_area(geom):
    '''Area from cell's UTM zone'''
    if (-180 <= geom.centroid.x <= 180) and (-90 <= geom.centroid.y <= 90):
        utm_epsg = convert_wgs_to_utm(geom.centroid.x, geom.centroid.y) 
        project = pyproj.Transformer.from_crs('EPSG:4236', f'EPSG:{utm_epsg}', always_xy=True).transform
        area = transform(project, geom).area
    else:
        print(f'invalid centroid {geom.centroid}')
        area = None
    return area

In [3]:
grid = get_s2_cells(8)

In [4]:
grid = create_s2_geometry(grid)

In [16]:
grid['lambert_perimeter'] = grid['geometry'].apply(get_perimeter_from_lambert)

In [13]:
grid

,cell_id,geometry,ind_lambert_area
0,00001,"POLYGON ((-45.00000 -35.26439, -44.70061 -35.4...",8.530865e+08
1,00003,"POLYGON ((-44.70061 -35.40491, -44.39929 -35.5...",8.586376e+08
2,00005,"POLYGON ((-44.70061 -35.12264, -44.39929 -35.2...",8.642055e+08
3,00007,"POLYGON ((-45.00000 -34.98262, -44.70061 -35.1...",8.585566e+08
4,00009,"POLYGON ((-45.00000 -34.70000, -44.70061 -34.8...",8.639809e+08
...,...,...,...
393211,bfff7,"POLYGON ((-44.69864 -35.68907, -44.39532 -35.8...",8.641845e+08
393212,bfff9,"POLYGON ((-45.00000 -35.54715, -44.69864 -35.6...",8.586777e+08
393213,bfffb,"POLYGON ((-45.30136 -35.68907, -45.00000 -35.8...",8.643263e+08
393214,bfffd,"POLYGON ((-45.60071 -35.54510, -45.30136 -35.6...",8.586757e+08


In [2]:
fiona.listlayers(r'd:\UTCloud\DGGS\grids\grids.gpkg')

['DGGRID_ISEA4H_7',
 'DGGRID_ISEA4T_7',
 'DGGRID_ISEA3H_9',
 'rhpix_5',
 'DGGRID_ISEA7H_5',
 's2_8',
 'H3_4',
 's2_8_clean',
 'rhpix_5_anomalies',
 'rhpix_5_clean',
 'DGGRID_ISEA7H_5_clean',
 'DGGRID_ISEA7H_5_anomalies',
 'DGGRID_ISEA3H_9_clean',
 'DGGRID_FULLER_4T',
 'DGGRID_FULLER_7H',
 'EAGGR4T_7',
 'DGGRID_ISEA4D_7',
 'DGGRID_ISEA4D_7_clean',
 'DGGRID_FULLER_4D',
 'DGGRID_FULLER_4D_clean']

In [3]:
grid = gpd.read_file(r'd:\UTCloud\DGGS\grids\grids.gpkg', layer='H3_4', driver='gpkg').dropna()

In [59]:
grid['isea_area'] = grid.to_crs('proj=isea')['geometry'].area

In [5]:
grid['ind_lambert_area'] = grid['geometry'].apply(get_lambert_area)

In [60]:
grid['goode_area'] = grid.to_crs('proj=goode')['geometry'].area

In [7]:
grid['ind_UTM_area'] = grid['geometry'].apply(get_utm_area)

In [ ]:
grid['ind_lambert_area'] = grid['geometry'].apply(get_lambert_area)

In [17]:
gpd.GeoDataFrame(grid).to_file(r'd:\UTCloud\DGGS\grids\grids.gpkg', layer='s2_8', driver='GPKG')